### CUSTOMER SEGMENTATION
---

### Libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import itertools

### Reading data

In [5]:
df = pd.read_csv("data.csv", index_col=0, encoding="ISO-8859-1")

In [9]:
df.head(2)

,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
InvoiceNo,,,,,,,
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### Processing data

In [11]:
df.dtypes

StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [12]:
df_correct_type = df.astype({
    "StockCode": "str",
    "Description": "str",
    "InvoiceDate": "datetime64[ns]",
    "Country": "str"
})

In [20]:
print("Number of duplicated rows: {0}".format(df_correct_type.duplicated().sum()))

Number of duplicated rows: 5848


In [21]:
df_correct_type.drop_duplicates(inplace=True)